# Question Answering

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings

persist_directory = "docs/Chroma"
embedding = OllamaEmbeddings(model = "nomic-embed-text")
vectordb = Chroma(persist_directory=persist_directory, embedding_function = embedding)


In [ ]:
print(vectordb._collection.count())

In [ ]:
question = "What is the major topics for this class?"
docs = vectordb.similarity_search(question, k=3)
len(docs)

In [ ]:
from langchain_community.llms import Ollama

llm = Ollama(
    model = "llama3"
)

### RetrievalQA chain

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever = vectordb.as_retriever()
)

In [ ]:
result = qa_chain({"query": question})
result['result']

### Prompt

In [ ]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [ ]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [ ]:
question = "Is probability a class topic?"
result = qa_chain({"query:", question})

In [ ]:
result["result"]

In [ ]:
result["source_documents"][0]

### RetrievalQA chain types

In [ ]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever = vectordb.as_retriever(),
    chain_type = "map_reduce"
)

In [ ]:
result = qa_chain_mr({"query": question})
result["result"]

In [ ]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGSMITH_API_KEY")

In [ ]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever = vectordb.as_retriever(),
    chain_type = "map_reduce"
)
result = qa_chain_mr({"query": question})
result["result"]

In [ ]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever = vectordb.as_retriever(),
    chain_type = "map_reduce"
)
result = qa_chain_mr({"query": question})
result["result"]

### RetrievalQA limitations
 
QA fails to preserve conversational history.

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [ ]:
question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]

In [ ]:
question = "why are those prerequesites needed?"
result = qa_chain({"query": question})
result["result"]